__Websites__<br>
    preprocessing methods outline - https://blog.ml.cmu.edu/2020/08/31/2-data-exploration/<br>
    outliers simple - https://analyticsindiamag.com/how-to-detect-and-treat-outliers-in-categorical-data/<br>
    pca collinearity removal/heatmap - https://towardsdatascience.com/how-do-you-apply-pca-to-logistic-regression-to-remove-multicollinearity-10b7f8e89f9b#:~:text=PCA%20in%20action%20to%20remove%20multicollinearity&text=PCA%20(Principal%20Component%20Analysis)%20takes,effectively%20eliminate%20multicollinearity%20between%20features.<br>
    vif - https://towardsdatascience.com/how-to-remove-multicollinearity-using-python-4da8d9d8abb2<br>
    pca 2D - https://towardsdatascience.com/principal-component-analysis-pca-from-scratch-in-python-7f3e2a540c51<br>
    pca 3D - https://drzinph.com/pca-visualized-with-3d-scatter-plots/<br>
    tsne -https://builtin.com/data-science/tsne-python<br>
    umap - https://blog.ml.cmu.edu/2020/08/31/2-data-exploration/<br>
    automatic outlier detection - https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/<br>
    3D t-sne and umap -  https://plotly.com/python/t-sne-and-umap-projections/#project-data-into-3d-with-tsne-and-pxscatter3d<br>
    knn - https://medium.datadriveninvestor.com/k-nearest-neighbors-in-python-hyperparameters-tuning-716734bc557f<br>
    hyperparameter tuning - https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74<br>
    model evaluation - https://medium.com/@MohammedS/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b<br>
    pydot and graphviz - https://github.com/WillKoehrsen/Data-Analysis/blob/master/random_forest_explained/Random%20Forest%20Explained.ipynb<br>

    To do:
    -Explore hyperparameter tuning time and accuracy graphs
    -look into other simple plots
    -Compare metrics of models
    -Confusion matrix and other evaluation models https://towardsdatascience.com/20-popular-machine-learning-metrics-part-1-classification-regression-evaluation-metrics-1ca3e282a2ce

In [ ]:
#sklearn imports

from sklearn import metrics, model_selection, ensemble
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import export_graphviz

#data handling imports
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# visualisation imports
# separated to reduce import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import umap.umap_ as umap

from statsmodels.stats.outliers_influence import variance_inflation_factor

import plotly.express as px

import pydot

from mlxtend.plotting import plot_decision_regions


__Preprocessing and Data Exploration__<br>
We want to predict edibility<br>
It can take values edible, poisonous or unknown. Unknown and poisonous are grouped together

First read the file and format it as desired

Missing values are stored as '?' characters so we convert to NaN to make the data easier to process

In [ ]:
#cell to read raw data

#read contents of data file
file = pd.read_csv("agaricus-lepiota.data", header = None)

# create list to be used as headers
features = ['edibility', 'capShape', 'capSurface', 'capColor', 'bruises', 'odor', 
            'gillAttachment', 'gillSpacing', 'gillSize', 'gillColor', 
            'stalkShape', 'stalkRoot', 'stalkSurfaceAboveRing', 'stalkSurfaceBelowRing', 
            'stalkColorAboveRing', 'stalkColorBelowRing', 'veilType', 'veilColor', 
            'ringNumber', 'ringType', 'sporePrintColor', 'population', 'habitat']

#convert '?' to NaN
file.replace({'?': np.nan}, inplace=True)

#converting data frame to csv
file.to_csv("Data/agaricus-lepiota.csv", header=features, index=False)

data = pd.read_csv("Data/agaricus-lepiota.csv")

#result = data.head(10)
#print(data)
#print(result)

Simple data exploration for illustration<br>
Shows that the only feature containing missing values is 'stalkRoot', with 2480 null values

It's impractical to perform univariate analysis on every single feature due to the high dimensionality, but we can at least illustrate some

In [ ]:
print(data.head(5)) # print first 5 rows
print(data.shape) # number of rows and columns
print(data.info()) # data types of each feature
print(data.isnull().sum()) # show how many NaN values in each feature

sns.set_theme(style="whitegrid")

# show even split of values in the feature to be predicted
data['edibility'] = LabelEncoder().fit_transform(data['edibility'])
fig = plt.figure(figsize=(20,15))
sns.countplot(x=data['edibility'])
plt.xlabel("Count")
plt.ylabel("Edibility")
plt.title("Countplot to Show number of Edible (0) and Poisonous (1) data points")
fig.savefig("Images/Edibility Countplot.png")

# this visualisation works best for features with a wide range of possible values
# chose 3 features as a starting point

f = plt.figure(figsize=(20,4))
f.add_subplot(1,3,1)
sns.countplot(x = data['population'])
sns.color_palette("pastel")
f.add_subplot(1,3,2)
sns.countplot(x = data['ringNumber'])
sns.color_palette("pastel")
f.add_subplot(1,3,3)
sns.countplot(x=data['gillColor'])
sns.color_palette("pastel")
f.savefig("Images/3 Countplots.png")

First need to use label encoding - OneHot encoding not applicable due to the high dimensionality of the dataset. OneHot Encoding would require using a large amount of memory

In [ ]:
#cell for encoding the data

#use label encoding on all features that are categorical
le = LabelEncoder()

for i in range(len(features)):
   try:
      data[features[i]] = le.fit_transform(data[features[i]])
   except:
      pass

data.to_csv("Data/agaricus-lepiota-encoded.csv")

In [ ]:
#violin plots to show distribution
figure, axes = plt.subplots(3,7,figsize = (30,20))
figure.suptitle("Violin Plots of feature scompared with Edibility")

graph_features = ['capShape', 'capSurface', 'capColor', 'bruises', 'odor', 
            'gillAttachment', 'gillSpacing', 'gillSize', 'gillColor', 
            'stalkShape', 'stalkRoot', 'stalkSurfaceAboveRing', 'stalkSurfaceBelowRing', 
            'stalkColorAboveRing', 'stalkColorBelowRing', 'veilColor', 
            'ringNumber', 'ringType', 'sporePrintColor', 'population', 'habitat']

count = 0
for i in range(0,3):
    for j in range(0,7):
        ax = sns.violinplot(ax = axes[i,j], data = data, x="edibility", y = graph_features[count])
        count +=1

Deletion methods of dealing with missing values<br>
**Only use one of the 2 cell below at a time**

**Don't use either if you're using the cell to predict missing values**

1. Delete any column containing missing values
2. Delete any row containing missing values

Since only one feature contains missing values (stalkRoot) method 1 is most effective as it deletes less data

In [ ]:
#1.
#removes column with missing values which in our data is only stalkRoot
#one way to deal with missing values
print(data.shape)

treated_data = data.drop('stalkRoot', axis = 1).reset_index(drop=True)

features.remove("stalkRoot")
print(features)

# show how much data is lost
print(treated_data.shape)

#convert dataframe to csv
treated_data.to_csv("Data/agaricus-lepiota-no-stalkRoot.csv")

#print(data.head(10))

In [ ]:
#2.
#removes all rows that have missing values
print(data.shape)

treated_data = data[data['stalkRoot'] != 4].reset_index(drop=True)
 
# show how much data is lost
print(treated_data.shape)

#convert dataframe to csv
treated_data.to_csv("Data/agaricus-lepiota-NaN-removed.csv")

Better way to deal with missing values: Use classifier or regressor based on values which aren't missing to predict the missing values<br>
Need to be careful to avoid overfitting (as with any predictor)

Could also set the values to some default or mean value


In [ ]:
#predict missing values for stalkRoot

#drop 'edibility' from dataframe as it will be the target feature in the later models
#It would be a bad idea to use 'edibility' to predict missing 'stalkRoot' values then use those values to later predict 'edibility'
NaN_predict = data.drop('edibility', axis = 1).reset_index(drop=True)

#split into set that contains missing values for 'stalkRoot' and set that does not contan missing values for 'stalkRoot'
#The 'NaN' values are encoded to be '4' so they are removed
NaN_train_data = NaN_predict[NaN_predict['stalkRoot']!= 4].copy()
NaN_test_data = NaN_predict[NaN_predict['stalkRoot'] == 4].copy()

#split data missing values into train and test set
NaN_X_train = NaN_train_data.drop('stalkRoot', axis = 1).reset_index(drop=True)
NaN_y_train = NaN_train_data['stalkRoot']

NaN_X_test = NaN_test_data.drop('stalkRoot', axis = 1).reset_index(drop=True)

###############################################################################################################################
#This code further breaks the data down into training and test sets comprising of rows that have no missing values
#Allows us to run models and check how accurately they can predict the missing values then choose an appropriate one
#In this case both Random Forest and Support Vector Classification predict that exact same values for 'stalkRoot' so either can be used
#Both have 100% accuracy in predicitng 'stalkRoot' so we know when we predict the missing values they will most likely be correct.

#Split data with non missing values into train and test set
X_train, X_test, y_train, y_test = train_test_split(NaN_train_data, NaN_train_data['stalkRoot'], test_size=0.2,random_state=42)

#create classes for both classification models
rfr = RandomForestClassifier()
svm = SVC()

#fit and predict both models
rfr.fit(X_train, y_train)
rfr_pred = rfr.predict(X_test)

svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

rfr_rounded = (np.rint(rfr_pred)).astype(int)

svm_rounded = (np.rint(svm_pred)).astype(int)

#create a list and append all predicted values which arent the same
different = []

for i in range(len(rfr_rounded)):
    if rfr_rounded[i] != svm_rounded[i]:
        different.append(rfr_rounded[i], svm_rounded[i])

#check how well models performed
print(len(different))

print(classification_report(y_test,rfr_rounded))

print(classification_report(y_test,svm_rounded))
##############################################################################################################################
#now this code is to predict missing values

#create random forest class
rfc = RandomForestClassifier()

#fit and predict data
rfc.fit(NaN_X_train,NaN_y_train)
y_pred = rfc.predict(NaN_X_test)

#set NaN values to be the predicted values
rounded_data = (np.rint(y_pred)).astype(int)
NaN_test_data['stalkRoot'] = rounded_data

NaN_test_data.to_csv("Data/predicted-missing-values.csv")

#combine the two dataframes to make the complete dataset
frames = [NaN_train_data, NaN_test_data]
treated_data = pd.concat(frames)
treated_data = treated_data.sort_index()
treated_data.insert(0, column = 'edibility', value = data['edibility'])

treated_data.to_csv("Data/full-data.csv")

In [ ]:

#create multiple datasets which can be preprocessed differently and compared later
baseline_X = data.drop('edibility', axis = 1).reset_index(drop=True)
baseline_y = data['edibility']

X = treated_data.drop('edibility', axis = 1).reset_index(drop=True)
y = treated_data['edibility'] 

graph_labels = []
for i in y:
    if i == 0:
        graph_labels.append("Edible")
    else:
        graph_labels.append("Poisonous")

#scale the data for correct visualisation
data_scaled = StandardScaler().fit_transform(X)

Now deal with outliers<br>
We define outliers as values a category can take which is significantly more or less frequent than the others<br>

Detect outliers based on relative frequency and remove any rows containing outliers in at least one category<br>
Then show histogram of data distribution within categories containing outliers

In [ ]:
#cell to detect and remove outliers

#decides relative frequency threshold, below which values are considered outliers
threshold = 0.01
contains_outliers = []
outlier_features = []
num_values = data.nunique()

print("Original dataset:",treated_data.shape)

#loop through categories and identify which contain outliers according to the above definitions
for i in range(len(features)):
    if num_values[i] > 2:
        value_frequencies = treated_data[features[i]].value_counts(normalize=True) #normalize parameter converts values to relative frequency

        #loop through features that contain outliers and remove the all rows that contain the outlier values
        for j in range(len(value_frequencies)):
            if value_frequencies.iloc[j] < threshold:
                treated_data = treated_data[treated_data[features[i]] != value_frequencies.index[j]]
                contains_outliers.append(value_frequencies)
                outlier_features.append(features[i])

#print(outlier_features)
#print(contains_outliers)

print("Outliers removed:",treated_data.shape)

#now display histogram
sns.set_theme(style = None)

if len(contains_outliers) != 0:
    for x in range(len(contains_outliers)):
        outlier_histogram = plt.hist(contains_outliers[x], alpha = 0.5, label = outlier_features[x])

    plt.legend(loc='upper right',prop={'size': 6})
    plt.xlabel("Relative frequency")
    plt.ylabel("Encoded label")
    plt.title("Histograms showing the relative frequency of attributes within each feature")
    plt.savefig("Images/Frequency Histogram.png")

Now check for any features which only contain one value<br>
These won't affect the model performance so we can remove them to simplify the dataset

In [ ]:
#cell to remove redundant features

num_values = treated_data.nunique()
#print(num_values)
removed_features = []


#treated_data = treated_data[treated_data[features[i]] != value_frequencies.index[j]]

j = 0
for i in range(len(features)):
    if num_values[i] == 1:
        treated_data = treated_data.drop(features[j],axis=1).reset_index(drop=True)
        removed_features.append(features[j])
        features.remove(features[j])
    else:
        j += 1

X = treated_data.drop('edibility', axis = 1).reset_index(drop=True) #update treated X
y = treated_data['edibility'] #update treated y

print("Removed",removed_features)
#print(treated_data.nunique())
#data = data.drop('veilType', axis = 1).reset_index(drop=True)
#features.remove('veilType')

Display heatmap to explore correlation between features<br>
Multicollinearity can lead to overfitting so need to remove correlated features

In [ ]:
#cell that uses variance inflation factor to identify and remove highly correlated data

#create a heatmap showing feature correlation
fig = plt.figure(figsize=(25, 20))
sns.heatmap(X.corr(), annot=True)
plt.savefig("Images/Heatmap.png")

#as you can see from heatmap there are some features with high correlation

#create dataframe with all features and their VIF
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
						for i in range(len(X.columns))]

print(vif_data)

features_to_drop = []

#loop through the features and add those with VIF higher than 7 to a list
for i in range(len(X.columns)):
	if vif_data["VIF"][i] > 5:
		drop = vif_data["VIF"][i]
		features_to_drop.append(vif_data["feature"][i])

#create new dataframe identified features removed
uncorrelated_X = X.drop(features_to_drop, axis = 1).reset_index(drop=True)

#scale for visualisation
uncorrelated_X_scaled = StandardScaler().fit_transform(uncorrelated_X)

cleaned_features = uncorrelated_X.columns

#generally features with VIF > 5 get removed but we can experiment with slightly higher i.e > 7

#create heatmap to compare feature correlation of new dataframe
fig2 = plt.figure(figsize=(25, 20))
sns.heatmap(uncorrelated_X.corr(), annot=True)
plt.savefig("Images/Heatmap 2.png")

#as seen by second heatmap these have much less colinearality

__Data Visualisation__

1. PCA<br>
Linear method so less complex than T-SNE, meaning it takes much less time<br>
But not so useful to visualise our dataset as shown by the explained variance and scree plot<br>

However it could be useful to reduce multicollinearity instead of data visualisation<br>
Could use the first 8 pc's to get about ~90% explained variance, 9 to get ~93%, 10 to get ~95% which greatly reduces the dimesionality of out dataset while mostly maintaining important information

In [ ]:
#data visualisation pca 2D
pca = PCA(random_state=42)
pca_features = pca.fit_transform(data_scaled)

#calculate percieved variance
per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
print(per_var)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

plt.rcParams["figure.figsize"] = (12,8)

#to be seen on a 2D-plot we can only use PC1 and PC2 which gives us the highest explained variance but still very low
#around 19% + 12% = 31% explained variance which is bad
plt.bar(x= range(1,len(per_var)+1), height = per_var, tick_label = labels)
plt.title('Scree Plot to Show the Explained Variance of Each PC')
plt.xlabel("PCA NO.")
plt.ylabel("Explained Variance (%)")
plt.show()
plt.savefig("Images/PCA_distribution")

plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

pca_df = pd.DataFrame({'pca_1': pca_features[:,0], 'pca_2': pca_features[:,1], 'Edibility': graph_labels})
print(pca_df)

#plot pca 1 and pca 2
sns.scatterplot(x = 'pca_1', y = 'pca_2', hue = 'Edibility', data = pca_df)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title("2D Visualisation using PCA")
plt.show()
plt.savefig("Images/PCA/2D PCA")

In [ ]:
#PCA 3D
pca = PCA(n_components = 3, random_state=42)
pca_features = pca.fit_transform(data_scaled)

#calculate percieved variance
per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
print(per_var)
print(pca_features)

treated_data["edibility"] = treated_data["edibility"].astype(str)

#plot PCA 3D
pca_fig = px.scatter_3d(
    pca_features, x=0, y=1, z=2,
    color=graph_labels, labels={'color': 'Edibility'},
    title = "Dimensionality reduction using PCA to visualise the dataset"
)
pca_fig.update_traces(marker_size=8)

pca_fig.update_layout(scene = dict(
                    xaxis_title='PCA 1',
                    yaxis_title='PCA 2',
                    zaxis_title='PCA 3')
)
pca_fig.show()

pca_fig.write_html("Images/PCA/PCA-3D.html")

2. T-SNE<br>
Unlike pca, it's not linear so much more complex and better for our data<br>
The result of t-sne is greatly affected by the 'perplexity' hyperparameter<br>
Try perplexity values in the range 5-100 default is 30. Higher perplexity takes longer but may be better

In [ ]:
#data visualisation t-sne 2D
n_components = 2
tsne = TSNE(n_components, perplexity=50, random_state = 42)
tsne_features = tsne.fit_transform(data_scaled)

tsne_df = pd.DataFrame({'tsne_1': tsne_features[:,0], 'tsne_2': tsne_features[:,1], 'label': treated_data['edibility']})

#plot tnse 1 and tsne 2
sns.scatterplot(x = 'tsne_1', y = 'tsne_2', hue = 'label', data = tsne_df)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.title("2D Visualisation using t-SNE")
plt.show()
plt.savefig("Images/t-SNE/2D t-SNE")

In [ ]:
#3D t-SNE using plotly
n_components = 3

#create t-SNE class and set hyperparameters
tsne = TSNE(n_components, perplexity=50, random_state=42)
tsne_features = tsne.fit_transform(data_scaled)

treated_data["edibility"] = treated_data["edibility"].astype(str)

#plot t-SNE
tsne_fig = px.scatter_3d(
    tsne_features, x=0, y=1, z=2,
    color=graph_labels, labels={'color': 'Edibility'},
    title = "Dimensionality reduction using t-SNE to visualise the dataset"
)
tsne_fig.update_traces(marker_size=8)

tsne_fig.update_layout(scene = dict(
                    xaxis_title='t-SNE 1',
                    yaxis_title='t-SNE 2',
                    zaxis_title='t-SNE 3')
)
tsne_fig.show()

tsne_fig.write_html("Images/t-SNE/t-SNE-3D.html")

3. UMAP <br>
UMAP works very similarly to tsne but gets more efficient for higher volumes of data <br>
Result greatly affected by n_neighbors and min_dist hyperparameters <br>
For n_neighbors try range 5-100 and min_dist 0.1-1

In [ ]:
#data visualisation UMAP

#create UMAP class and set hyperparameters
fit = umap.UMAP(
    n_neighbors=50,
    min_dist=0.25,
    n_components=2,
    metric = 'correlation',
    random_state = 42
)

umap_features = fit.fit_transform(data_scaled)

umap_df = pd.DataFrame({'umap_1': umap_features[:,0], 'umap_2': umap_features[:,1], 'Edibility': graph_labels})


#plot 2D UMAP
sns.scatterplot(x = 'umap_1', y = 'umap_2', hue = 'Edibility', data = umap_df)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.title("2D Visualisation using UMAP")
plt.show()
plt.savefig("Images/UMAP/2D UMAP")


In [ ]:
#UMAP 3D using plotly

#create UMAP class and set hyperparameters
fit = umap.UMAP(
    n_neighbors=75,
    min_dist=0.5,
    n_components=3,
    metric = 'correlation',
    random_state = 42
)

umap_features = fit.fit_transform(data_scaled)

treated_data["edibility"] = treated_data["edibility"].astype(str)

#plot UMAP 3D
umap_fig = px.scatter_3d(
    umap_features, x=0, y=1, z=2,
    color=graph_labels, labels={'color': 'Edibility'},
    title = "Dimensionality reduction using UMAP to visualise the dataset"
)
umap_fig.update_traces(marker_size=5)

umap_fig.update_layout(scene = dict(
                    xaxis_title='UMAP 1',
                    yaxis_title='UMAP 2',
                    zaxis_title='UMAP 3')
)

umap_fig.show()

umap_fig.write_html("Images/UMAP/UMAP-3D.html")


__Models__ 

Compare baseline data (baseline_X and baseline_y) with treated data (uncorrelated_X and y)

1. K-nearest neighbours<br>
Hyperparameter: n_neighbors<br>
Low value less accurate, high value overfits

In [ ]:
# knn hyperparameter exploration: accuracy v n_neighbors

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2, random_state=42)
neighbours = range(1,30)
accuracies = []

for i in range(len(neighbours)):
    knn = KNeighborsClassifier(n_neighbors=neighbours[i])
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

plt.plot(neighbours, accuracies)
plt.title("Knn accuracy as n_neighbors increases")
plt.xlabel("Num neighbours")
plt.ylabel("Accuracy")
plt.savefig("Images/KNN/Knn accuracy v n_neighbors.png")

In [ ]:
# knn hyperparameter exploration: decision regions
# plot decision regions as the accuracy changes to help explain why

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2, random_state=42)
n_neighbours = [1, 2, 5, 11, 12, 15, 25] # points on the accuracy graph with changes

for i in range(len(n_neighbours)):
    knn = KNeighborsClassifier(n_neighbors=n_neighbours[i])
    knn.fit(X_train, y_train)

    for j in range(2):
        plt.subplot(2,1,j+1)
        plot_decision_regions(X_test.values, y_test.values, clf = knn, legend = 2, feature_index = [0,2], filler_feature_values={1: j}, filler_feature_ranges={1:0.75})
        plt.title("n_neighbors="+str(n_neighbours[i])+" gillSpacing="+str(j))
        plt.xlabel("capSurface")
        plt.ylabel("habitat")
    
    plt.tight_layout()
    plt.savefig("Images/KNN/Knn Decision Regions "+str(n_neighbours[i])+".png")
    plt.close()

In [ ]:
#cell for  knn model

# hyperparameters
n_neighbors = list(range(1,30))
weights = ['uniform','distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
leaf_size = list(range(1,50))
p = [1,2]

# create dictionary
random_grid = {'n_neighbors': n_neighbors,
               'weights': weights,
               'algorithm': algorithm,
               'leaf_size': leaf_size,
               'p': p}

# baseline model
knn_baseline = KNeighborsClassifier()

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)
knn_baseline.fit(X_train, y_train)
baseline_knn_pred = knn_baseline.predict(X_test)

# random hyperparameter values
knn_random = RandomizedSearchCV(estimator = knn_baseline, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = 'accuracy')

knn_random.fit(X_train, y_train)

best_random_values = knn_random.best_params_
print("\n", best_random_values, "\n")
best_random_knn = knn_random.best_estimator_
print(best_random_knn)

#predict using best random model and get model accuracy report
best_random_pred = best_random_knn.predict(X_test)

#create a refined hyperparameter grid with values close to the best random estimator to look for further optimization
# fits may fail if n_neighbors or leaf_size < 15 however they will just be skipped so this is okay
param_grid = {
    'n_neighbors': list(range((best_random_values['n_neighbors']-20),(best_random_values['n_neighbors']+20))),
    'weights': [best_random_values['weights']],
    'algorithm': [best_random_values['algorithm']],
    'leaf_size': list(range((best_random_values['leaf_size']-20),(best_random_values['leaf_size']+20))),
    'p': [best_random_values['p']]
}

knn_refined = KNeighborsClassifier()

# find the best scoring model using the refined hyperparameter grid
knn_grid = GridSearchCV(estimator = knn_refined, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1, scoring = 'accuracy')

knn_grid.fit(X_train, y_train)

print("\n", knn_grid.best_params_, "\n")
best_grid = knn_grid.best_estimator_
print(best_grid)

# predict with refined model
best_refined_pred = best_grid.predict(X_test)

fig,(ax1,ax2) = plt.subplots(1,2)

#show model accuracy report
print("Baseline\n")
print(classification_report(y_test,baseline_knn_pred))

cf_matrix = confusion_matrix(y_test, baseline_knn_pred)

print(cf_matrix)

sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = 'Blues', annot = True, ax = ax1)

print("Random Search\n")
print(classification_report(y_test,best_random_pred))

cf_matrix_2 = confusion_matrix(y_test, best_random_pred)

print(cf_matrix_2)

#sns.heatmap(cf_matrix_2/np.sum(cf_matrix_2), cmap = 'Blues', annot = True, ax = ax2)

print("Grid Search\n")
print(classification_report(y_test,best_refined_pred))

cf_matrix_3 = confusion_matrix(y_test, best_refined_pred)

print(cf_matrix_3)

sns.heatmap(cf_matrix_3/np.sum(cf_matrix_3), cmap = 'Blues', annot = True, ax = ax2)

In [ ]:
#cell for RandomForestClassifier

#create distribution of chosen hyperparamaters to be used in random search
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [3, 5, 10]
min_samples_leaf = [2, 4, 6]
bootstrap = [True, False]

labels = ('True Negative', 'False Positive', 'Fale Negative', 'True Positive')

#create dictionary with distribution of chosen hyperparameters
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#basline model to compare to when optimising
rf = RandomForestClassifier()

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)

baseline_rfc = RandomForestClassifier(random_state = 42, n_estimators = 10)

baseline_rfc.fit(X_train, y_train)
baseline_rfc_pred = baseline_rfc.predict(X_test)


#find best scoring random hyperparameter combination
rfc_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = 'accuracy')

rfc_random.fit(X_train, y_train)
best_random_values = rfc_random.best_params_

print("\n", rfc_random.best_params_, "\n")
best_random_rfc = rfc_random.best_estimator_
print(best_random_rfc)

#predict using best random model and get model accuracy report
best_pred = best_random_rfc.predict(X_test)

#create a refined hyperparameter grid with values close to the best random estimator to look for further optimization
param_grid = {
    'n_estimators': list(range((best_random_values['n_estimators']-20),(best_random_values['n_estimators']+20))),
    'max_features': [best_random_values['max_features']],
    'max_depth': list(range((best_random_values['max_depth']-9),(best_random_values['max_depth']+10))),
    'min_samples_split': list(range((best_random_values['min_samples_split']-1),(best_random_values['min_samples_split']+1))),
    'min_samples_split': list(range((best_random_values['min_samples_leaf']-1),(best_random_values['min_samples_leaf']+1))),
    'bootstrap': [best_random_values['bootstrap']]
}

rf2 = RandomForestClassifier()

#find the best scoring model using the refined hyperparameter grid
rfc_grid = GridSearchCV(estimator = rf2, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1, scoring = 'accuracy')

rfc_grid.fit(X_train, y_train)

print("\n", rfc_grid.best_params_, "\n")
best_grid = rfc_grid.best_estimator_
print(best_grid)

#predict values using this model and show accuracy to compare models later
best_pred2 = best_grid.predict(X_test)

fig,(ax1,ax2) = plt.subplots(1,2)

#show model accuracy report
print("Baseline\n")
print(classification_report(y_test,baseline_rfc_pred))

cf_matrix = confusion_matrix(y_test, baseline_rfc_pred)

print(cf_matrix)

sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = 'Blues', annot = True, ax = ax1)

print("Random Search\n")
print(classification_report(y_test,best_pred))

cf_matrix_2 = confusion_matrix(y_test, best_pred)

print(cf_matrix_2)

#sns.heatmap(cf_matrix_2/np.sum(cf_matrix_2), cmap = 'Blues', annot = True, ax = ax2)

print("Grid Search\n")
print(classification_report(y_test,best_pred2))

cf_matrix_3 = confusion_matrix(y_test, best_pred2)

print(cf_matrix_3)

sns.heatmap(cf_matrix_3/np.sum(cf_matrix_3), cmap = 'Blues', annot = True, ax = ax2)

#get one tree from estimators
tree = best_grid.estimators_[5]

#Export the image to a dot file then use file to creat a graph
export_graphviz(tree, out_file = 'tree.dot', feature_names = cleaned_features, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('Images/tree.png')

In [ ]:
# SVC gamma hyperparameter exploration

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)

gammas = [0.1,1,10,100]
kernels = ['rbf', 'poly', 'linear']
cs = [0.1,1,10,100]
degrees = [0,1,2,3,4]

for gamma in gammas:
    svc_gamma = SVC(gamma = gamma, random_state = 42)
    svc_gamma.fit(X_train,y_train)

    for i in range(2):
        plt.subplot(2,1,i+1)
        plot_decision_regions(X_test.values, y_test.values, clf = svc_gamma, legend = 2, feature_index = [0,2], filler_feature_values={1: i}, filler_feature_ranges={1:0.75})
        title = "gamma = " + str(gamma) + " gillSpacing="+str(i)
        print(title)
        plt.title(title)
        plt.xlabel("capSurface")
        plt.ylabel("habitat")

    image_directory = "Images/SVC Regions/gamma = " + str(gamma) + ".png"
    print(image_directory)
    plt.tight_layout()
    plt.savefig(image_directory)
    plt.close()

In [ ]:
# SVC kernels hyperparameter exploration

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)

kernels = ['rbf', 'poly', 'linear']

for kernel in kernels:
    svc_kernel = SVC(kernel = kernel, random_state = 42)
    svc_kernel.fit(X_train,y_train)

    for i in range(2):
        plt.subplot(2,1,i+1)
        plot_decision_regions(X_test.values, y_test.values, clf = svc_kernel, legend = 2, feature_index = [0,2], filler_feature_values={1: i}, filler_feature_ranges={1:0.75})
        title = "kernel = " + str(kernel) + " gillSpacing="+str(i)
        print(title)
        plt.title(title)
        plt.xlabel("capSurface")
        plt.ylabel("habitat")

    image_directory = "Images/SVC Regions/kernel = " + str(kernel) + ".png"
    print(image_directory)
    plt.tight_layout()
    plt.savefig(image_directory)
    plt.close()

In [ ]:
# SVC C hyperparameter exploration

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)

cs = [0.1,1,10,100]

for c in cs:
    svc_c = SVC(C = c, random_state = 42)
    svc_c.fit(X_train,y_train)

    for i in range(2):
        plt.subplot(2,1,i+1)
        plot_decision_regions(X_test.values, y_test.values, clf = svc_c, legend = 2, feature_index = [0,2], filler_feature_values={1: i}, filler_feature_ranges={1:0.75})
        title = "C = " + str(c) + " gillSpacing="+str(i)
        print(title)
        plt.title(title)
        plt.xlabel("capSurface")
        plt.ylabel("habitat")

    image_directory = "Images/SVC Regions/C = " + str(c) + ".png"
    print(image_directory)
    plt.tight_layout()
    plt.savefig(image_directory)
    plt.close()

In [ ]:
#cell for SVC

#create distribution of chosen hyperparamaters to be used in SVC
gamma = [20,50,100,200,400]
kernel = ['linear', 'rbf']
c = [20,50,100,200,400]

#create dictionary with distribution of chosen hyperparameters
random_grid = {'gamma': gamma,
               'kernel': kernel,
               'C': c,
}

print(random_grid)

#basline model to compare to when optimising
svc = SVC(random_state=42,)

X_train, X_test, y_train, y_test = train_test_split(uncorrelated_X, y, test_size=0.2)

baseline_SVC = SVC(random_state = 42)

baseline_SVC.fit(X_train, y_train)
baseline_SVC_pred = baseline_SVC.predict(X_test)

#find best scoring random hyperparameter combination
SVC_random = RandomizedSearchCV(estimator = svc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = 'accuracy')

SVC_random.fit(X_train, y_train)
best_random_values = SVC_random.best_params_

print("\n", SVC_random.best_params_, "\n")
best_random_SVC = SVC_random.best_estimator_
print(best_random_SVC)

#predict using best random model and get model accuracy report
best_pred = best_random_SVC.predict(X_test)

#create a refined hyperparameter grid with values close to the best random estimator to look for further optimization

param_grid = {
    'gamma': list(range((best_random_values['gamma']-10),(best_random_values['gamma']+10))),
    'kernel': [best_random_values['kernel']],
    'C': list(range((best_random_values['C']-10),(best_random_values['C']+10))),
}

svc2 = SVC(random_state = 42)

#find the best scoring model using the refined hyperparameter grid
svc_grid = GridSearchCV(estimator = svc2, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1, scoring = 'accuracy')

svc_grid.fit(X_train, y_train)

print("\n", svc_grid.best_params_, "\n")
best_grid = svc_grid.best_estimator_
print(best_grid)

#predict values using this model and show accuracy to compare models later
best_pred2 = best_grid.predict(X_test)

print("\n", svc_grid.best_params_, "\n")
best_grid = svc_grid.best_estimator_
print(best_grid)

#predict values using this model and show accuracy to compare models later
best_pred2 = best_grid.predict(X_test)

fig,(ax1,ax2) = plt.subplots(1,2)

#show model accuracy report
print("Baseline\n")
print(classification_report(y_test,baseline_SVC_pred))

cf_matrix = confusion_matrix(y_test, baseline_SVC_pred)

print(cf_matrix)

#sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = 'Blues', annot = True, ax = ax1)

print("Random Search\n")
print(classification_report(y_test,best_pred))

cf_matrix_2 = confusion_matrix(y_test, best_pred)

print(cf_matrix_2)

#sns.heatmap(cf_matrix_2/np.sum(cf_matrix_2), cmap = 'Blues', annot = True, ax = ax2)

print("Grid Search\n")
print(classification_report(y_test,best_pred2))

cf_matrix_3 = confusion_matrix(y_test, best_pred2)

print(cf_matrix_3)
